# Dependency

In [3]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [4]:
%matplotlib inline

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)
#Remove before deployment

In [6]:
spotify_df = pd.read_csv(r'C:\Users\Bugalia\musicRec\tracks.csv')

In [7]:
spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [8]:
data_w_genres = pd.read_csv(r'C:\Users\Bugalia\musicRec\artists.csv')

In [9]:
data_w_genres.head()

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0


In [10]:
pd.isnull(spotify_df).sum()

id                   0
name                71
popularity           0
duration_ms          0
explicit             0
artists              0
id_artists           0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
dtype: int64

In [11]:
pd.isnull(data_w_genres).sum()

id             0
followers     13
genres         0
name           0
popularity     0
dtype: int64

In [12]:
data_w_genres.dtypes

id             object
followers     float64
genres         object
name           object
popularity      int64
dtype: object

In [13]:
data_w_genres['genres_upd'] = data_w_genres['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [14]:
spotify_df['artists_upd_v1'] = spotify_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
spotify_df['artists_upd_v2'] = spotify_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
spotify_df['artists_upd'] = np.where(spotify_df['artists_upd_v1'].apply(lambda x: not x), spotify_df['artists_upd_v2'], spotify_df['artists_upd_v1'] )

In [15]:
spotify_df['artists_song'] = spotify_df.apply(lambda row: str(row['artists_upd'][0])+str(row['name']), axis = 1)

In [16]:
spotify_df.sort_values(['artists_song','release_date'], ascending = False, inplace = True)

In [17]:
spotify_df[spotify_df['name']=='Adore You']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,0,-5.407,1,0.0315,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus],[],[Miley Cyrus],Miley CyrusAdore You
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,8,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You
92524,1M4qEo4HE3PRaCOM7EXNJq,Adore You,74,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-06,0.676,0.771,8,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You


In [18]:
spotify_df.drop_duplicates('artists_song',inplace = True)

In [19]:
spotify_df[spotify_df['name']=='Adore You']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,0,-5.407,1,0.0315,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus],[],[Miley Cyrus],Miley CyrusAdore You
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,8,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You


In [20]:
artists_exploded = spotify_df[['artists_upd','id']].explode('artists_upd')

In [21]:
artists_exploded_enriched = artists_exploded.merge(data_w_genres, how = 'left', left_on = 'artists_upd',right_on = 'name')
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres_upd.isnull()]

In [22]:
artists_exploded_enriched_nonnull[artists_exploded_enriched_nonnull['id_x'] =='6KuQTIu1KoTTkLXKrwlLPV']

,artists_upd,id_x,id_y,followers,genres,name,popularity,genres_upd
193829,Robert Schumann,6KuQTIu1KoTTkLXKrwlLPV,2UqjDAXnDxejEyE0CzfUrZ,423826.0,"['classical', 'early romantic era', 'german ro...",Robert Schumann,64.0,"[classical, early_romantic_era, german_romanti..."
193830,Vladimir Horowitz,6KuQTIu1KoTTkLXKrwlLPV,4Ws5hSoABAwvGJ4LhHwHgq,92365.0,"['classical', 'classical performance', 'classi...",Vladimir Horowitz,54.0,"[classical, classical_performance, classical_p..."


In [23]:
artists_genres_consolidated = artists_exploded_enriched_nonnull.groupby('id_x')['genres_upd'].apply(list).reset_index()

In [24]:
artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['genres_upd'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [25]:
artists_genres_consolidated.head()

,id_x,genres_upd,consolidates_genre_lists
0,0004Uy71ku11n3LMpuyf59,[[polish_rock]],[polish_rock]
1,000CSYu4rvd8cQ7JilfxhZ,"[[country_quebecois, rock_quebecois]]","[country_quebecois, rock_quebecois]"
2,000DsoWJKHdaUmhgcnpr8j,[[barnmusik]],[barnmusik]
3,000G1xMMuwxNHmwVsBdtj1,"[[candy_pop, new_wave, new_wave_pop, permanent...","[rock, new_wave, power_pop, new_wave_pop, cand..."
4,000KblXP5csWFFFsD6smOy,"[[chamame, folclore_salteno, folklore_argentino]]","[folklore_argentino, folclore_salteno, chamame]"


In [26]:
spotify_df.dtypes

id                   object
name                 object
popularity            int64
duration_ms           int64
explicit              int64
artists              object
id_artists           object
release_date         object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
artists_upd_v1       object
artists_upd_v2       object
artists_upd          object
artists_song         object
dtype: object

In [27]:
artists_exploded_enriched.dtypes

artists_upd     object
id_x            object
id_y            object
followers      float64
genres          object
name            object
popularity     float64
genres_upd      object
dtype: object

In [28]:
artists_genres_consolidated.rename(columns = {'id_x':'id'}, inplace = True)

In [29]:
spotify_df = spotify_df.merge(artists_genres_consolidated[['id','consolidates_genre_lists']], on = 'id', how = 'left')

In [30]:
spotify_df.tail()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
523470,3YCC4oCzagcfx4s3fp3XFz,Bandolera,35,322926,1,"[""At' Fat"", 'Yemil', 'Boza', 'El Tachi']","['455JBMYIT1lWf1Djj4rDc7', '7g4cPtKxNx146qdaY9...",2020-11-17,0.502,0.814,5,-2.688,0,0.1330,0.524,0.00000,0.1510,0.6720,72.344,4,"[ Fat"", , , , , ]",[At' Fat],"[ Fat"", , , , , ]","Fat"", Bandolera",NaN
523471,5sr38tHMSPpaENiNcjUpFi,Cathrina,10,184560,0,"[""Henry D' Cruz"", ""Helen D'Souza""]","['3nH2WLSi1MhvlmJNkbLbIv', '4NR8lI9YoPWo5v2Kv6...",1976-01-01,0.661,0.677,1,-8.447,0,0.1160,0.683,0.00154,0.0361,0.8090,116.855,4,"[ Cruz"", ""Helen D]","[Henry D' Cruz, Helen D'Souza]","[ Cruz"", ""Helen D]","Cruz"", ""Helen DCathrina",NaN
523472,6Pu2leLWEfThVIqJilw6O9,The Passion of Our Lord according to St. Matth...,4,79027,0,"[""Boys' Choir of The Church of The Transfigura...","['47EwjzhfZKotjVtvlQFNFS', '5H155SiWINLkYRElqw...",1963,0.177,0.208,11,-22.988,0,0.0388,0.988,0.94600,0.0982,0.0587,132.053,3,"[ Choir of The Church of The Transfiguration"",...",[Boys' Choir of The Church of The Transfigurat...,"[ Choir of The Church of The Transfiguration"",...","Choir of The Church of The Transfiguration"", ...",NaN
523473,7f092ECp06XbFS6Ms5Yk6R,Mia Oraia Petalouda,26,100313,0,"[""Typaldos' Children's Choire""]",['7JJrBciIAQBPOBB5U6u49p'],2014-06-08,0.773,0.183,0,-9.370,1,0.0370,0.956,0.00000,0.1190,0.8640,79.950,4,[ Children],[Typaldos' Children's Choire],[ Children],ChildrenMia Oraia Petalouda,NaN
523474,1yw3fl6UGrxjq3BTeJh6xz,I Koukouvagia,28,96508,0,"[""Typaldos' Children's Choire""]",['7JJrBciIAQBPOBB5U6u49p'],2014-06-08,0.748,0.256,9,-10.177,1,0.0532,0.969,0.00000,0.1200,0.9240,179.898,4,[ Children],[Typaldos' Children's Choire],[ Children],ChildrenI Koukouvagia,NaN


In [31]:
spotify_df['year'] = spotify_df['release_date'].apply(lambda x: x.split('-')[0])

In [32]:
float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values

In [33]:
ohe_cols = 'popularity'

In [34]:
spotify_df['popularity'].describe()

count    523475.000000
mean         27.235870
std          18.030233
min           0.000000
25%          13.000000
50%          27.000000
75%          40.000000
max          99.000000
Name: popularity, dtype: float64

In [35]:
spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [36]:
spotify_df.rename(columns = {'consolidates_genre_lists_x':'consolidates_genre_lists'}, inplace = True)

In [37]:
spotify_df['consolidates_genre_lists'] = spotify_df['consolidates_genre_lists'].apply(lambda d: d if isinstance(d, list) else [])

In [38]:
spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,year,popularity_red
0,3u1C6nWVRoP5F0w8gGrDL3,사랑의 미로,25,222380,0,['최진희'],['1NSrAf8XJYJVgAXKoxaMet'],1987-06-01,0.367,0.194,7,-19.057,1,0.0400,0.617,0.000006,0.1620,0.367,144.316,4,[최진희],[],[최진희],최진희사랑의 미로,[trot],1987,5
1,1Mv4u308L16NZDZiD6HZCy,사랑은 힘든가봐,28,213440,0,['지수'],['4c9QIMfEbIIynuaswyxGx9'],2005-12-23,0.675,0.785,4,-5.026,0,0.0280,0.379,0.000000,0.3530,0.623,103.008,4,[지수],[],[지수],지수사랑은 힘든가봐,[],2005,5
2,1jvoY322nxyKXq8OBhgmSY,어떡하죠,44,244360,0,['지선'],['2Mo9NQaNCFCWSR5CnlfmbN'],2011-10-13,0.606,0.341,0,-7.094,1,0.0513,0.779,0.000000,0.1440,0.294,135.667,4,[지선],[],[지선],지선어떡하죠,[],2011,8
3,2ghebdwe2pNXT4eL34T7pW,그아픔까지사랑한거야,32,237688,0,['조정현'],['2WTpsPucygbYRnCnoEUkJQ'],1989-06-15,0.447,0.215,10,-16.478,1,0.0272,0.568,0.000001,0.0649,0.177,71.979,4,[조정현],[],[조정현],조정현그아픔까지사랑한거야,[],1989,6
4,7rxpWwcXNgDUXl0wN0gUvp,천국의 기억 장정우 Version,31,280372,0,['장정우'],['5L7zKs2ftwENWOMI7LFaN1'],2003-12-24,0.494,0.656,7,-6.347,0,0.0262,0.659,0.000007,0.1110,0.420,82.003,4,[장정우],[],[장정우],장정우천국의 기억 장정우 Version,[],2003,6


In [39]:
def ohe_prep(df, column, new_name):
    tf_df = pd.get_dummies(df[column])
    features_names  = tf_df.columns
    tf_df.columns = [new_name + '|' + str(col) for col in features_names]
    tf_df.reset_index(drop = True, inplace = True)
    return tf_df

In [40]:
def create_feature_set(df, float_cols):
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    year_ohe = ohe_prep(df, 'year','year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)

    final['id'] = df['id'].values

    return final

In [41]:
complete_feature_set = create_feature_set(spotify_df, float_cols=float_cols)

MemoryError: Unable to allocate 18.6 GiB for an array with shape (4765, 523475) and data type float64

# Connection to Spotify

In [42]:
client_id = 'a636353dbb784d53a11894ca416b233b'
client_secret = '4144a7268b36445aba0ac4dcb29567f7'

In [43]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [44]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [45]:
token = util.prompt_for_user_token(scope, client_id=client_id, client_secret=client_secret, redirect_uri='http://localhost:8081/')

In [46]:
sp = spotipy.Spotify(auth=token)

In [47]:
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [48]:
id_name

{'depressed boi hours': '5wab9fqDNrt6piHis90O0E',
 'i miss her A LOT': '1ZeXl9PaxK23p7iBkhkw13',
 'hype party': '4X12OgkUqWs9kHyTMrj0Zk',
 'HIndie': '3FJynN8hvOhvZWCJSPALSm',
 'LoseYourself': '6oejRM3KxPUp9yaOq4TFY8',
 '!(FallInLove)': '1Ip5k5nJ19nFxtYtZxosKq',
 'KKindie': '2UjxAnjAE2HJyQQl9sqX8C',
 'GetLost': '2QKUNKegP1VxHiv74pzXp7',
 'RapRapRap': '1fbiEv6ShsivSvMu31UhKS',
 'KKock': '6fzssZ4K1V45agpIifB2x9',
 'FapToJap': '4dJGSLT9H7WJR8PLsoUQo0'}

In [49]:
def create_necessary_outputs(playlist_name,id_dic, df):
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [50]:
id_name

{'depressed boi hours': '5wab9fqDNrt6piHis90O0E',
 'i miss her A LOT': '1ZeXl9PaxK23p7iBkhkw13',
 'hype party': '4X12OgkUqWs9kHyTMrj0Zk',
 'HIndie': '3FJynN8hvOhvZWCJSPALSm',
 'LoseYourself': '6oejRM3KxPUp9yaOq4TFY8',
 '!(FallInLove)': '1Ip5k5nJ19nFxtYtZxosKq',
 'KKindie': '2UjxAnjAE2HJyQQl9sqX8C',
 'GetLost': '2QKUNKegP1VxHiv74pzXp7',
 'RapRapRap': '1fbiEv6ShsivSvMu31UhKS',
 'KKock': '6fzssZ4K1V45agpIifB2x9',
 'FapToJap': '4dJGSLT9H7WJR8PLsoUQo0'}

In [51]:
playlist_GetLost = create_necessary_outputs('GetLost', id_name,spotify_df)

In [52]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_songs(df):    
    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(len(temp) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['name'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [53]:
playlist_GetLost

,artist,name,id,url,date_added
64,Asha Bhosle,Kisi Nazar Ko Tera Intezar,04Erz9shwLnOL55apcDEAG,https://i.scdn.co/image/ab67616d00001e029820c9...,2022-02-21 04:29:55+00:00
63,Lata Mangeshkar,Tere Bina Zindagi Se,57l24qsMzRsvecxRM0tfC6,https://i.scdn.co/image/ab67616d00001e02102904...,2021-12-16 11:14:54+00:00
62,Hemant Kumar,Jane Woh Kaise Log The,1KaNtSpU12ZFyTPc07gTht,https://i.scdn.co/image/ab67616d00001e029841c2...,2021-12-11 21:42:16+00:00
61,Jagjit Singh,Pyar Mujh Se Jo Kiya,7yJAOX3aLb7EWfHFDyn26L,https://i.scdn.co/image/ab67616d00001e02ce6618...,2021-12-02 23:42:42+00:00
56,Kishore Kumar,Aanewala Pal Janewala Hai,3Tel7yJVZrPclGFsMsxZcV,https://i.scdn.co/image/ab67616d00001e0221fd20...,2021-11-14 15:40:07+00:00
55,Anup Ghoshal,Tujhse Naraz Nahin Zindagi - Anup Ghoshal,4Sbubs41C1LaukO72ioZvj,https://i.scdn.co/image/ab67616d00001e0254f105...,2021-10-19 23:28:39+00:00
52,Lata Mangeshkar,Tujhse Naraz Nahin Zindagi - Lata Mangeshkar,4M6QtwT7kFCMqztO4E96kk,https://i.scdn.co/image/ab67616d00001e0254f105...,2021-09-21 02:04:49+00:00
51,Kishore Kumar,Kuchh To Log Kahenge,0ophwoCMQqMF8hOSkklmjL,https://i.scdn.co/image/ab67616d00001e022f83c1...,2021-08-24 20:04:21+00:00
49,Geeta Dutt,Hum Aap Ki Ankhon Mein,6xAiuvSFPKgPbSkj7hupoP,https://i.scdn.co/image/ab67616d00001e029841c2...,2021-08-19 21:24:08+00:00
48,Jagjit Singh,Tum Ko Dekha To Yeh Khayal Aaya,66KBRmenGEJkTKtbTWl7tS,https://i.scdn.co/image/ab67616d00001e02ce6618...,2021-08-19 21:22:30+00:00


In [ ]:
visualize_songs(playlist_GetLost)

In [55]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [ ]:
complete_feature_set_playlist_vector_GetLost, complete_feature_set_nonplaylist_GetLost = generate_playlist_feature(complete_feature_set, playlist_GetLost, 1.09)

In [ ]:
complete_feature_set_playlist_vector_GetLost

In [ ]:
def generate_playlist_recos(df, features, nonplaylist_features):    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [ ]:
getLost_top40 = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector_GetLost, complete_feature_set_nonplaylist_GetLost)

In [ ]:
getLost_top40

In [ ]:
visualize_songs(getLost_top40)